In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi
import csv
import os

In [2]:
class KaggleLoadSubmit:
    def __init__(self, competition_name, csv_dir):
        """Kaggleのデータのダウンロードおよび提出を行うクラス

        Args:
            competition_name (string): Kaggleのコンペ名
            csv_dir (string): ダウンロード先と、提出先のcsvのディレクトリをstringで指定
        """
        self.kaggle_api = KaggleApi()
        self.csv_dir = csv_dir
        self.competition_name = competition_name
        if self.csv_dir[-1] != '\\':
            self.csv_dir += '\\'
        self.kaggle_api.authenticate()

    def download_zip(self):
        """コンペティションのzipファイルをダウンロードする
        """
        ret = self.kaggle_api.competition_download_files(self.competition_name, self.csv_dir, force=True)
        print(ret)

    def submit_csv(self, csv_name, message):
        """csvデータを提出する

        Args:
            csv_name (string): csvの名前
            message (string): 提出メッセージ
        """
        csv_path = self.csv_dir + csv_name
        self.kaggle_api.competition_submit(csv_path, message, self.competition_name)

In [3]:
kls = KaggleLoadSubmit('titanic', 'C:\\Users\\Takanori\\Desktop\\Kaggle\\titanic\\data')

# Titanic
https://www.kaggle.com/c/titanic/

In [4]:
import numpy as np
import pandas as pd

In [5]:
# データの読み込み
data_dir = 'C:\\Users\\Takanori\\Desktop\\Kaggle\\titanic\\data'
train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'test.csv'))

def fix(data):
    # 明らかに要らないデータを削除
    delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
    data.drop(delete_columns, axis=1, inplace=True)
    # 文字列を数値に
    data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
    data['Embarked'].fillna(('S'), inplace=True)
    data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
    data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
    data['Age'].fillna(data['Age'].median(), inplace=True)
    data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
    data['IsAlone'] = 0
    data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

for data in [train, test]:
    fix(data)


In [10]:
# LightGBM
# 1.データを学習用と検証用に分割
X_train = train.drop('Survived', axis=1)
Y_train = train['Survived']
X_test = test[:]

from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.3, random_state=0, stratify=Y_train)

In [11]:
# 2. カテゴリ変数（文字列等）を列挙
categorical_features = ['Embarked', 'Pclass', 'Sex']

In [ ]:
# 3.ハイパーパラメータ調整
import optuna
from sklearn.metrics import log_loss
import lightgbm as lgb

def objective(trial):
    params = {
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': 0.05,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    lgb_train = lgb.Dataset(X_train, Y_train,
                                             categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_valid, Y_valid, reference=lgb_train,
                                            categorical_feature=categorical_features)

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(Y_valid, y_pred_valid)
    return score

study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

In [16]:
study.best_params

{'max_bin': 390, 'num_leaves': 101}

In [ ]:
# 4.predict
import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, Y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, Y_valid, reference=lgb_train, categorical_feature=categorical_features)

params = {'objective': 'binary',
          'max_bin': study.best_params['max_bin'],
          'learning_rate': 0.05,
          'num_leaves': study.best_params['num_leaves']}

model = lgb.train(params, lgb_train,
                  valid_sets=[lgb_train, lgb_eval],
                  verbose_eval=10,
                  num_boost_round=1000,
                  early_stopping_rounds=10)

Y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [18]:
# 結果を01に
Y_pred = (Y_pred > 0.5).astype(int)

In [19]:
# save_csv
sub = pd.read_csv(os.path.join(data_dir, 'gender_submission.csv'))
sub['Survived'] = Y_pred
sub.to_csv(os.path.join(data_dir, 'my_submission.csv'), index=False)

In [20]:
kls.submit_csv(os.path.join('my_submission.csv'), '0316')

100%|██████████| 3.18k/3.18k [00:04<00:00, 807B/s]
